In [2]:
import sys,os
sys.path.append('../')
from R import load_features_layer
from R.models import *
from R.data import *
from R.paths import *
import pyFCI
import numpy as np
from time import time 

start = time()

crop_size = 28
class_id = 0
layer_id = 0
dbg=1

model_id = 0
model_name,W_model = model_list[model_id]
print(f'{model_name=}')
layer_names = layers_dict[model_name][:] 
class_list = list(class_dict.keys())[:]

layer_name = layer_names[layer_id]
print(f'layer {layer_name}')

key = 'shuffled'
print(f'{key=}')

model_name='resnet18'
layer layer1.0.relu_1
key='shuffled'


In [7]:
### loading activations:
i0 = 0
i_max = 79
chunk_size = 100
flatten_activations = 1

afolder = get_afolder(model_name,key,crop_size)
print(afolder)

/scratch/sacevedo/Imagenet2012/act/resnet18/shuffled/crop_size28/


In [8]:
a = load_features_layer(afolder,
                      layer_name,
                      chunk_size,
                      i0=i0,
                      i_max=i_max,
                      flatten=flatten_activations,
                      )
Ns,N = a.shape
EDfile = get_EDfilename('.',model_name,layer_name,key)

: 

In [ ]:
np.savetxt(fname=EDfile,X=a.shape,fmt='%d')

### BINARIZATION
precision = 8
a = np.round(a,precision)
a = 2*np.sign(a).astype(int)-1

### SEPARATING DATA IN BLOCKS TO HAVE THE SAME NUMBER OF DATA SAMPLES AS THE FIRST EXPERIMENT
n_blocks = len(class_list) # there were 7 classes, so lets divide the dataset in 7 again
block_size = Ns // n_blocks
print(f'{block_size=}')
print(f'{Ns=}')
a = a[class_id*block_size:(class_id+1)*block_size]
print(f'{a.shape=}')

### FCI
a = pyFCI.center_and_normalize(a)
fci = pyFCI.FCI(a)
d,x0,err = pyFCI.fit_FCI(fci)

### EXPORTING
resultsfolder = makefolder(base=f'results/FCI/shuffle/',
                           create_folder=True,
                           crop_size=crop_size,
                           key=class_id,
                           layer_id=layer_id,
                           )
np.savetxt(resultsfolder + 'FCI.txt',X=[d,x0,err])

print(f'this took {(time()-start)/60:.1f} mins')